### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2022, 1, 11)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2022-01-11'



,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
0,BCH,21.40,0,"5,000","15,000",21.30,2022-01-06,2022-01-11,3,20.90,19.80,-11,21.00,26.75,26.41,"107,000.00",I
1,BGRIM,42.00,0,"3,000","9,000",40.00,2021-12-30,2022-01-11,8,40.50,39.00,-6,40.75,56.00,53.00,"126,000.00",I
12,EPG,12.90,24H,"3,000","12,000",11.20,2022-01-10,2022-01-11,1,12.20,12.40,2,12.60,13.70,14.74,"38,700.00",I
2,GLOBAL,21.00,0,"7,500","7,500",20.00,2022-01-06,2022-01-11,3,19.90,20.80,9,21.30,24.70,27.08,"157,500.00",I
15,IMH,20.70,0,"3,000","9,000",18.00,2022-01-10,2022-01-11,1,19.70,18.20,-15,20.60,16.80,19.20,"62,100.00",I
13,IP,23.20,0,"3,500","10,500",20.20,2022-01-10,2022-01-11,1,19.20,19.30,1,19.40,24.47,26.90,"81,200.00",I
3,IVL,50.50,12,"3,000","12,000",42.00,2022-01-06,2022-01-11,3,45.25,47.25,8,47.75,50.00,54.34,"151,500.00",I
7,KBANK,150.00,0,800,"2,400",136.50,2022-01-07,2022-01-11,2,142.50,143.50,2,144.00,153.00,175.20,"120,000.00",I
9,NER,7.50,0,"12,000","36,000",7.45,2022-01-07,2022-01-11,2,7.00,7.40,8,7.45,8.65,10.60,"90,000.00",I
14,RJH,36.25,0,"1,500","4,500",33.00,2022-01-10,2022-01-11,1,33.50,33.25,-1,33.50,39.00,36.50,"54,375.00",I


In [5]:
my_array = df_stock.to_numpy()
my_array

array([['BCH', 21.4, 0, 5000, 15000, 21.3, '2022-01-06', '2022-01-11', 3,
        20.9, 19.8, -11, 21.0, 26.75, 26.41, 107000.0, 'I'],
       ['BGRIM', 42.0, 0, 3000, 9000, 40.0, '2021-12-30', '2022-01-11',
        8, 40.5, 39.0, -6, 40.75, 56.0, 53.0, 126000.0, 'I'],
       ['GLOBAL', 21.0, 0, 7500, 7500, 20.0, '2022-01-06', '2022-01-11',
        3, 19.9, 20.8, 9, 21.3, 24.7, 27.08, 157500.0, 'I'],
       ['IVL', 50.5, 12, 3000, 12000, 42.0, '2022-01-06', '2022-01-11',
        3, 45.25, 47.25, 8, 47.75, 50.0, 54.34, 151500.0, 'I'],
       ['SYNEX', 33.0, 12, 1500, 6000, 31.5, '2021-12-30', '2022-01-11',
        8, 36.5, 32.0, -18, 35.75, 36.5, 28.67, 49500.0, 'I'],
       ['TU', 20.9, 12, 6000, 18000, 20.8, '2021-12-30', '2022-01-11', 8,
        19.5, 20.7, 12, 20.9, 23.0, 26.67, 125399.99999999999, 'I'],
       ['WHART', 12.8, 0, 10000, 10000, 12.5, '2021-12-28', '2022-01-11',
        10, 12.8, 12.1, -7, 12.8, 14.08, 15.98, 128000.0, 'I'],
       ['KBANK', 150.0, 0, 800, 2400, 136.5,

In [6]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([16, 12,  2, 13,  4,  2,  6, 13,  7,  2, 12,  8,  5, 39, 12, 25])

In [7]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,BCH,21.40,0,"5,000","15,000",21.30,2022-01-06,2022-01-11,3,20.90,19.80,-11,21.00,26.75,26.41,"107,000.00",I,16
1,BGRIM,42.00,0,"3,000","9,000",40.00,2021-12-30,2022-01-11,8,40.50,39.00,-6,40.75,56.00,53.00,"126,000.00",I,12
2,GLOBAL,21.00,0,"7,500","7,500",20.00,2022-01-06,2022-01-11,3,19.90,20.80,9,21.30,24.70,27.08,"157,500.00",I,2
3,IVL,50.50,12,"3,000","12,000",42.00,2022-01-06,2022-01-11,3,45.25,47.25,8,47.75,50.00,54.34,"151,500.00",I,13
4,SYNEX,33.00,12,"1,500","6,000",31.50,2021-12-30,2022-01-11,8,36.50,32.00,-18,35.75,36.50,28.67,"49,500.00",I,4
5,TU,20.90,12,"6,000","18,000",20.80,2021-12-30,2022-01-11,8,19.50,20.70,12,20.90,23.00,26.67,"125,400.00",I,2
6,WHART,12.80,0,"10,000","10,000",12.50,2021-12-28,2022-01-11,10,12.80,12.10,-7,12.80,14.08,15.98,"128,000.00",I,6
7,KBANK,150.00,0,800,"2,400",136.50,2022-01-07,2022-01-11,2,142.50,143.50,2,144.00,153.00,175.20,"120,000.00",I,13
8,SAT,24.25,0,"2,500","7,500",21.10,2022-01-07,2022-01-11,2,23.40,23.50,1,23.70,23.90,27.75,"60,625.00",I,7
9,NER,7.50,0,"12,000","36,000",7.45,2022-01-07,2022-01-11,2,7.00,7.40,8,7.45,8.65,10.60,"90,000.00",I,2


In [8]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
10,BCH,"15,000",21.30,"319,500.00",0.33,"4,950.00",1.55,3
29,BGRIM,"9,000",40.00,"360,000.00",0.45,"4,050.00",1.12,4
19,CKP,"30,000",5.00,"150,000.00",0.04,"1,050.00",0.70,3
6,DIF,"60,000",14.70,"882,000.00",1.04,"62,640.00",7.10,2
12,DOHOME,"12,000",24.30,"291,600.00",0.10,"1,200.00",0.41,3
25,EPG,"12,000",11.20,"134,400.00",0.23,"2,760.00",2.05,3
8,GLOBAL,"7,500",20.00,"150,000.00",nan,nan,nan,3
17,HREIT,"30,000",8.95,"268,500.00",0.69,"20,796.00",7.75,2
27,IMH,"9,000",18.00,"162,000.00",nan,nan,nan,3
33,IP,"10,500",20.20,"212,100.00",0.18,"1,942.50",0.92,3


In [9]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,BCH,16,21.40,0,"5,000","15,000",21.30,20.90,19.80,-11,21.00,26.75,26.41,I,3,"107,000.00"
1,BGRIM,12,42.00,0,"3,000","9,000",40.00,40.50,39.00,-6,40.75,56.00,53.00,I,4,"126,000.00"
2,GLOBAL,2,21.00,0,"7,500","7,500",20.00,19.90,20.80,9,21.30,24.70,27.08,I,3,"157,500.00"
3,IVL,13,50.50,12,"3,000","12,000",42.00,45.25,47.25,8,47.75,50.00,54.34,I,3,"151,500.00"
4,SYNEX,4,33.00,12,"1,500","6,000",31.50,36.50,32.00,-18,35.75,36.50,28.67,I,3,"49,500.00"
5,TU,2,20.90,12,"6,000","18,000",20.80,19.50,20.70,12,20.90,23.00,26.67,I,4,"125,400.00"
6,WHART,6,12.80,0,"10,000","10,000",12.50,12.80,12.10,-7,12.80,14.08,15.98,I,4,"128,000.00"
7,KBANK,13,150.00,0,800,"2,400",136.50,142.50,143.50,2,144.00,153.00,175.20,I,4,"120,000.00"
8,SAT,7,24.25,0,"2,500","7,500",21.10,23.40,23.50,1,23.70,23.90,27.75,I,3,"60,625.00"
9,NER,2,7.50,0,"12,000","36,000",7.45,7.00,7.40,8,7.45,8.65,10.60,I,4,"90,000.00"


In [10]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
GLOBAL,2,21.00,0,7500,7500,20.00,19.90,20.80,9,21.30,24.70,27.08,I,3,157500.0
TU,2,20.90,12,6000,18000,20.80,19.50,20.70,12,20.90,23.00,26.67,I,4,125400.0
NER,2,7.50,0,12000,36000,7.45,7.00,7.40,8,7.45,8.65,10.60,I,4,90000.0
SYNEX,4,33.00,12,1500,6000,31.50,36.50,32.00,-18,35.75,36.50,28.67,I,3,49500.0
EPG,5,12.90,24H,3000,12000,11.20,12.20,12.40,2,12.60,13.70,14.74,I,3,38700.0
WHART,6,12.80,0,10000,10000,12.50,12.80,12.10,-7,12.80,14.08,15.98,I,4,128000.0
SAT,7,24.25,0,2500,7500,21.10,23.40,23.50,1,23.70,23.90,27.75,I,3,60625.0
SIS,8,44.50,24,1500,4500,40.50,44.50,42.50,-8,44.25,48.75,36.88,I,3,66750.0
BGRIM,12,42.00,0,3000,9000,40.00,40.50,39.00,-6,40.75,56.00,53.00,I,4,126000.0


In [11]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)